In [1]:
!pip install transformers

In [2]:
import pandas as pd
import json
import gzip
from pathlib import Path
from tqdm.notebook import tqdm
import torch
from transformers import BartTokenizer, BartModel
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import gdown
import urllib.request
import zipfile

from tqdm.notebook import tqdm
tqdm.pandas()

# Amazon data

In [8]:
AMAZON_REVIEWS = "amazon_reviews.json.gz"
AMAZON_ITEMS = "amazon_clothes.json.gz"

In [6]:
url = 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Clothing_Shoes_and_Jewelry_5.json.gz'
filename = AMAZON_REVIEWS
urllib.request.urlretrieve(url, filename)

('amazon_reviews.json.gz', <http.client.HTTPMessage at 0x1f585ed2cb0>)

In [7]:
url = 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Clothing_Shoes_and_Jewelry.json.gz'
filename = AMAZON_ITEMS
urllib.request.urlretrieve(url, filename)

('amazon_clothes.json.gz', <http.client.HTTPMessage at 0x1f585ed2920>)

In [3]:
products_metadata = dict()
wanted_category = "cardigan"
with gzip.open("amazon_clothes.json.gz", 'rb') as fp:
    i = []
    for line in tqdm(fp.readlines()):
        meta_line = json.loads(line)
        if wanted_category in map(str.lower, meta_line['category']):
            products_metadata.setdefault(meta_line['asin'], meta_line)
            i.append(meta_line)

  0%|          | 0/2685059 [00:00<?, ?it/s]

In [4]:
cardigan_items = pd.DataFrame(i)
cardigan_items

,category,description,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,price,also_buy,also_view,fit,details,main_cat
0,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...",[Long sleeve handkerchief cardigan with pleath...,Derek Heart Girls' L/SLV Handkerchief Cardigan...,Derek Heart,"[95% Polyester, 5% Spandex, Imported, Machine ...","3,405,822inClothing,ShoesJewelry(",5 star,B00079R2MS,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...",[Big girl cotton sweater cardigan with embroid...,Hartstrings Big Girls' Big Girl Sweater Cardig...,NaN,"[100% Cotton, Imported, Machine Wash, Front fl...","10,031,302inClothing,ShoesJewelry(",5 star,B000FIJ4TA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Clothing, Shoes & Jewelry, Women, Clothing, C...",[This cardigan style sweater has a tie front c...,Lilo Maternity Tie Front Sweater (Pink),Lilo Maternity,"[Cotton, long sleeves, pointelle design, cardi...","16,723,371inClothing,ShoesJewelry(",5 star5 star (0%),B0016OSWK4,NaN,NaN,$40.14 - $61.00,NaN,NaN,NaN,NaN,NaN
3,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...","[Big girl cotton sweater cardigan, button down...",Hartstrings Big Girls' Big Sweater Cardigan,NaN,"[100% Cotton, Imported, Machine Wash, 100-perc...","9,199,644inClothing,ShoesJewelry(",5 star,B001CJQQSS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...",NaN,Rare Editions Girl's Cotton Cardigan,Rare Editions,[Sleeveless bodice; Satin bow trim at waist; K...,"18,818,171inClothing,ShoesJewelry(",5 star,B001U1MDRG,NaN,NaN,$39.96,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...","[Long sleeve open front marled sweater cozy , ...",Derek Heart Big Girls' Long Sleeve Open Front ...,Derek Heart,"[100% Acrylic, Imported, Machine Wash, Dress, ...","1,258,383inClothing,ShoesJewelry(",5 star,B01H7S1WWW,NaN,NaN,NaN,"[B0134AB8XA, B016KX88S6, B074GZJZJJ, B01529O7I...",NaN,"class=""a-normal a-align-center a-spacing-smal...",NaN,NaN
1037,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",[This cardigan by Denim & Supply features wool...,Denim & Supply Ralph Lauren Womens Boyfriend W...,Polo Ralph Lauren,[23%Acrylic/22%Nylon/20%Wool/20%Mohair/15%Visc...,"11,453,690inClothing,ShoesJewelry(",5 star5 star (0%),B01H7V69UO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN
1038,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...",NaN,ZHUANNIAN Little Girls Solid Cardigan Sweaters,ZHUANNIAN,"[Button closure, Hand Wash Machine Washable., ...","773,151inClothing,ShoesJewelry(",5 star,B01HCWDN3Y,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$13.99,"[B01M8IBV1A, B01K49D3BE, B01KLCJOGM, B074RD697...","[B01M8IBV1A, B073CG5FWH, B071F17D9L, B074RD697...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN
1039,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...",[A pretty knit design and trendy fringe trim o...,Epic Threads Girls' Knit Fringe Sweater,Epic Threads,[Shipping Information:\n \n...,"4,498,462inClothing,ShoesJewelry(",5 star5 star (0%),B01HEAYIA6,NaN,NaN,$16.30 - $20.31,NaN,"[B07CHD5HKV, B01LW7T4FP, B0157QWYZU, B07G2YZBW...",NaN,NaN,NaN


In [6]:

asin_set = set(cardigan_items['asin'])
print(asin_set)

{'B00HGJNOKG', 'B00XRF5B2W', 'B008064WEQ', 'B005JENS24', 'B017IR7HJO', 'B01H42KG2I', 'B00E5AJSXQ', 'B00YGIFCF0', 'B014W70614', 'B010G8NXC2', 'B008B691BO', 'B00PIUC9PG', 'B01H7S1FB0', 'B00KA7K20Q', 'B01569B3SM', 'B00WW79JV0', 'B00YS7559C', 'B005CG0AJI', 'B003NGLL0G', 'B00F6OIVII', 'B00HRCGO6I', 'B00CWYW35C', 'B01H7S11UA', 'B005DTD6EA', 'B0086EM79E', 'B00YBQPSS8', 'B01DU4WK5U', 'B00EPPUU9M', 'B01E4X7J0C', 'B00CC65HDK', 'B00FKXFDGI', 'B00HC6FUZA', 'B00KU82XCK', 'B00ZQALBC0', 'B00DPEHJIO', 'B015QHR8I8', 'B013U6BUKO', 'B017H8H73U', 'B01H2HF93G', 'B00DPEHKP6', 'B0094N0ZLS', 'B01FYBZIX8', 'B00MFBUOA8', 'B00FXRHH2O', 'B00DZ0RCRG', 'B00OW9V28E', 'B00JZM2YP8', 'B00N3IM1AS', 'B01ENGOR3W', 'B004HHOFXK', 'B00400X8BO', 'B00YX6HHLW', 'B01HEAYIA6', 'B0189OZF76', 'B00KCMKM18', 'B00D3DFCQI', 'B0067ID9Z2', 'B005CTHR82', 'B0179RNO4U', 'B0163N5MZA', 'B014R1PMY6', 'B018WNN24W', 'B00GK9Z1RM', 'B019RIZAQY', 'B007FHZOX4', 'B00H8NEDXM', 'B003XIHUBI', 'B005JENS0Q', 'B00JZM37IG', 'B00JZM2DY0', 'B01H7S27IK', 'B00E

In [9]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  df = {}
  i = 0
  for d in parse(path):
    if d["asin"] in asin_set:
      df[i] = d
      i += 1
  return pd.DataFrame.from_dict(df, orient='index')

reviews = getDF(AMAZON_REVIEWS)

In [10]:
reviews

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,vote
0,5.0,True,"01 5, 2015",A2150DC5GKC9FL,B003NGLL0G,"{'Size:': ' Big Girls', 'Color:': ' Navy-ldda'}",Susan Alexander,My daughter goes to a charter school and they ...,Great Quality!,1420416000,NaN,NaN
1,5.0,True,"11 13, 2014",A1K1QAK7BBI4RS,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-he...",Tammy Smith,fits perfect!,Five Stars,1415836800,NaN,NaN
2,5.0,True,"10 13, 2014",A2GG708F913A7Z,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ld...",Theresa,"Recommend this seller,Has good clothes.","Very nice Sweater,",1413158400,NaN,NaN
3,4.0,True,"08 27, 2014",A1C4XG5FL8D1UC,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ld...",Rosanna,Very satisfied with the quality.,Four Stars,1409097600,NaN,NaN
4,5.0,True,"12 8, 2017",AJHPW0WZNVQQ0,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ah...",Erin Snyder,Cute little sweater. My daughter wears one nea...,Quality product,1512691200,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2124,5.0,True,"12 25, 2016",A354E34YDN9B3H,B01H7S23T8,"{'Size:': ' Big Girls', 'Color:': ' Cream'}",Amazon Customer,Awesome but the sleeves are tight.,Five Stars,1482624000,NaN,NaN
2125,3.0,True,"12 2, 2016",A2U9I0GKYC5ENM,B01H7S23T8,"{'Size:': ' Big Girls', 'Color:': ' Black Beau...",Ed Davin,Nice sweater. I bought if for a gift and I ho...,Nice sweater. I bought if for a gift and ...,1480636800,NaN,NaN
2126,5.0,True,"10 30, 2017",ASTHAOG1RBSWI,B01HCWDN3Y,"{'Size:': ' 3-4 Years', 'Color:': ' Grey'}",JMS,Perfect weight to wear over a shirt,Five Stars,1509321600,NaN,NaN
2127,5.0,True,"10 22, 2017",A6GOZT13KZ1FZ,B01HCWDN3Y,"{'Size:': ' 18-24 Months', 'Color:': ' Pink'}",Trina,Cute sweater. Perfect fit. Good quality for th...,Loveeee it,1508630400,NaN,NaN


In [11]:
pivot_table = reviews.pivot_table(index='reviewerID', columns='asin', values='overall', fill_value=0)
pivot_table

asin,B003L3L2ZA,B003NGLL0G,B004XY6T1S,B005DSLTUE,B005DSLU9Y,B005DTBBN8,B005DTD6EA,B005DTD6V8,B005GFUGBC,B005GFVKL2,...,B01ENGOG6A,B01ENGOR3W,B01ENL6VZO,B01ERHA9GG,B01H3OVIJC,B01H42KHWC,B01H7S11UA,B01H7S1WWW,B01H7S23T8,B01HCWDN3Y
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0604201MBJEJ93VKB77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0944456Z3LN62I2DT3O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A102OO0GRIR4NA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1037HGE7B63JX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103WTSUGJTUVL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZTYFU6FPB97B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZUO4VO2EGMI3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZUXN5C512DQB,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
for index, row in pivot_table.iterrows():
    for gameid, score in row.items():
        # Check if the score is not 0 and print the values
        if score != 0:
            print(f"User ID: {index}, Game ID: {gameid}, Score: {score}")


User ID: A0604201MBJEJ93VKB77, Game ID: B00ECW9P1W, Score: 5.0
User ID: A0944456Z3LN62I2DT3O, Game ID: B00NO5OTUU, Score: 5.0
User ID: A102OO0GRIR4NA, Game ID: B00NRR4SQ0, Score: 5.0
User ID: A102OO0GRIR4NA, Game ID: B019JMU496, Score: 5.0
User ID: A1037HGE7B63JX, Game ID: B00805IQO4, Score: 3.0
User ID: A103WTSUGJTUVL, Game ID: B00KU82XCK, Score: 5.0
User ID: A1043UVQVI4STF, Game ID: B00KCMMYEG, Score: 5.0
User ID: A109W7BH9ND707, Game ID: B01ENL6VZO, Score: 1.0
User ID: A10E3F50DIUJEE, Game ID: B003NGLL0G, Score: 4.0
User ID: A10G3ZQPM21P3J, Game ID: B00DPEHII0, Score: 5.0
User ID: A10G3ZQPM21P3J, Game ID: B00EH8SJHC, Score: 5.0
User ID: A10GD6IEW6NL8J, Game ID: B01569B3SM, Score: 5.0
User ID: A10KV05TC7JJSV, Game ID: B003L3L2ZA, Score: 5.0
User ID: A10MWXSSYGDOBS, Game ID: B0118XNFS6, Score: 5.0
User ID: A10O03WA13RUU0, Game ID: B005DTBBN8, Score: 4.0
User ID: A10QTM4IX095RA, Game ID: B00EH8SVU2, Score: 5.0
User ID: A10XULCDFD8VC7, Game ID: B00URG925Q, Score: 4.0
User ID: A110E9SIOB

In [13]:
filtered_reviews = reviews[pd.notna(reviews['reviewText'])]
filtered_reviews

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,vote
0,5.0,True,"01 5, 2015",A2150DC5GKC9FL,B003NGLL0G,"{'Size:': ' Big Girls', 'Color:': ' Navy-ldda'}",Susan Alexander,My daughter goes to a charter school and they ...,Great Quality!,1420416000,NaN,NaN
1,5.0,True,"11 13, 2014",A1K1QAK7BBI4RS,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-he...",Tammy Smith,fits perfect!,Five Stars,1415836800,NaN,NaN
2,5.0,True,"10 13, 2014",A2GG708F913A7Z,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ld...",Theresa,"Recommend this seller,Has good clothes.","Very nice Sweater,",1413158400,NaN,NaN
3,4.0,True,"08 27, 2014",A1C4XG5FL8D1UC,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ld...",Rosanna,Very satisfied with the quality.,Four Stars,1409097600,NaN,NaN
4,5.0,True,"12 8, 2017",AJHPW0WZNVQQ0,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ah...",Erin Snyder,Cute little sweater. My daughter wears one nea...,Quality product,1512691200,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2124,5.0,True,"12 25, 2016",A354E34YDN9B3H,B01H7S23T8,"{'Size:': ' Big Girls', 'Color:': ' Cream'}",Amazon Customer,Awesome but the sleeves are tight.,Five Stars,1482624000,NaN,NaN
2125,3.0,True,"12 2, 2016",A2U9I0GKYC5ENM,B01H7S23T8,"{'Size:': ' Big Girls', 'Color:': ' Black Beau...",Ed Davin,Nice sweater. I bought if for a gift and I ho...,Nice sweater. I bought if for a gift and ...,1480636800,NaN,NaN
2126,5.0,True,"10 30, 2017",ASTHAOG1RBSWI,B01HCWDN3Y,"{'Size:': ' 3-4 Years', 'Color:': ' Grey'}",JMS,Perfect weight to wear over a shirt,Five Stars,1509321600,NaN,NaN
2127,5.0,True,"10 22, 2017",A6GOZT13KZ1FZ,B01HCWDN3Y,"{'Size:': ' 18-24 Months', 'Color:': ' Pink'}",Trina,Cute sweater. Perfect fit. Good quality for th...,Loveeee it,1508630400,NaN,NaN


In [14]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDFcategory(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
    if i == 1000000:
      break
  return pd.DataFrame.from_dict(df, orient='index')

In [19]:
items = getDFcategory(AMAZON_ITEMS)

In [20]:
filtered_df = items[items['category'].apply(lambda x: 'cardigan' in x)]
filtered_df

,category,description,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,also_view,price,fit,also_buy,main_cat,tech1,details,similar_item,tech2
90446,"[Clothing, Shoes & Jewelry, Women, Clothing, C...",[This cardigan style sweater has a tie front c...,Lilo Maternity Tie Front Sweater (Pink),Lilo Maternity,"[Cotton, long sleeves, pointelle design, cardi...","16,723,371inClothing,ShoesJewelry(",5 star5 star (0%),B0016OSWK4,NaN,NaN,NaN,$40.14 - $61.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396872,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",NaN,Kensie Girl Juniors Space Dye Cardigan,Kensie Girl,"[100% Cotton, Hand Wash, Space dye novelty sti...","8,495,755inClothing,ShoesJewelry(",5 star,B005FISURM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969146,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",[This great back to school basic is great for ...,G2 Chic Women's Long Sleeve Light Weight Sweat...,G2 Chic,"[20% nylon, 80% COTTON 20% NYLON, cable knit, ...","1,818,459inClothing,ShoesJewelry(",5 star,B00E9ECXGC,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
with zipfile.ZipFile("cardigan_pics.zip","r") as zip_ref:
    zip_ref.extractall("amazon_cardigan_pics")

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])
model.to(device)
model.eval()

c:\Users\Iga Miller\OneDrive\Pulpit\Studia_mgr_SI\Semestr2\Sys_rek\l3n-wstep-do-projektu-team1\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Iga Miller\OneDrive\Pulpit\Studia_mgr_SI\Semestr2\Sys_rek\l3n-wstep-do-projektu-team1\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [74]:
import shutil
import os


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

root_dir = 'C:/Users/Iga Miller/OneDrive/Pulpit/Studia_mgr_SI/Semestr2/Sys_rek/l3n-wstep-do-projektu-team1/amazon_cardigan_pics/amazon_cardigan_pics'
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)

    if os.path.isdir(folder_path) and not os.listdir(folder_path):
        print(f"Removing empty folder: {folder_path}")
        shutil.rmtree(folder_path)

dataset = ImageFolder(root='C:/Users/Iga Miller/OneDrive/Pulpit/Studia_mgr_SI/Semestr2/Sys_rek/l3n-wstep-do-projektu-team1/amazon_cardigan_pics/amazon_cardigan_pics', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

Removing empty folder: C:/Users/Iga Miller/OneDrive/Pulpit/Studia_mgr_SI/Semestr2/Sys_rek/l3n-wstep-do-projektu-team1/amazon_cardigan_pics/amazon_cardigan_pics\B00ECW9P1W
Removing empty folder: C:/Users/Iga Miller/OneDrive/Pulpit/Studia_mgr_SI/Semestr2/Sys_rek/l3n-wstep-do-projektu-team1/amazon_cardigan_pics/amazon_cardigan_pics\B00ECW9VHU
Removing empty folder: C:/Users/Iga Miller/OneDrive/Pulpit/Studia_mgr_SI/Semestr2/Sys_rek/l3n-wstep-do-projektu-team1/amazon_cardigan_pics/amazon_cardigan_pics\B00ECWAAIO
Removing empty folder: C:/Users/Iga Miller/OneDrive/Pulpit/Studia_mgr_SI/Semestr2/Sys_rek/l3n-wstep-do-projektu-team1/amazon_cardigan_pics/amazon_cardigan_pics\B00EH8SJHC
Removing empty folder: C:/Users/Iga Miller/OneDrive/Pulpit/Studia_mgr_SI/Semestr2/Sys_rek/l3n-wstep-do-projektu-team1/amazon_cardigan_pics/amazon_cardigan_pics\B00EH8SVU2
Removing empty folder: C:/Users/Iga Miller/OneDrive/Pulpit/Studia_mgr_SI/Semestr2/Sys_rek/l3n-wstep-do-projektu-team1/amazon_cardigan_pics/amazon

In [76]:
root_dir = 'C:/Users/Iga Miller/OneDrive/Pulpit/Studia_mgr_SI/Semestr2/Sys_rek/l3n-wstep-do-projektu-team1/amazon_cardigan_pics/amazon_cardigan_pics'
subfolders = [f.name for f in os.scandir(root_dir) if f.is_dir()]
class_to_idx = {subfolder: idx for idx, subfolder in enumerate(subfolders)}
idx_to_class = {idx: class_name for class_name, idx in class_to_idx.items()}


def features_from_images(model, dataloader, idx_to_class):
    id_to_features_dict = {}

    for images, labels in dataloader:
        with torch.no_grad():
            features = model(images.to(device)).flatten(start_dim=1).cpu().numpy()

        game_ids = [idx_to_class[label.item()] for label in labels]
        for game_id, feature in zip(game_ids, features):
            if game_id in id_to_features_dict:
                id_to_features_dict[game_id].append(feature)
            else:
                id_to_features_dict[game_id] = [feature]
    flattened_data = [(game_id, sublist) for game_id, sublists in id_to_features_dict.items() for sublist in sublists]
    df = pd.DataFrame(flattened_data, columns=['asin', 'image_features'])
    return df


games_with_features = features_from_images(model, dataloader, idx_to_class)


In [77]:
games_with_features

,asin,image_features
0,B003NGLL0G,"[0.5060015, 0.53560853, 0.60426265, 1.0715923,..."
1,B003NGLL0G,"[1.914037, 1.586476, 0.59251326, 1.524742, 0.6..."
2,B003NGLL0G,"[1.3829921, 0.76654136, 0.6254153, 1.7982793, ..."
3,B003NGLL0G,"[1.602183, 1.7477553, 0.17946701, 0.8129126, 0..."
4,B003NGLL0G,"[0.9194918, 1.1864418, 0.18671028, 1.115975, 1..."
...,...,...
184,B01H3OVIJC,"[0.55591106, 0.0, 0.0511671, 2.8716004, 0.3315..."
185,B01H3OVIJC,"[3.0360045, 0.4813564, 0.015725618, 1.5883383,..."
186,B01H3OVIJC,"[3.0225108, 0.4713684, 1.2808826, 0.23879941, ..."
187,B01H3OVIJC,"[0.21762373, 0.017481118, 0.33292818, 1.724785..."


In [78]:
games_with_features = games_with_features.groupby(by="asin").agg({"image_features": lambda x: x.values.mean()})
games_with_features

,image_features
asin,
B003NGLL0G,"[1.264941, 1.1645646, 0.43767372, 1.2647002, 0..."
B005DSLTUE,"[2.276295, 0.8267818, 0.4272437, 1.5532677, 0...."
B005DSLU9Y,"[1.8207275, 0.6596862, 0.440968, 1.3359298, 0...."
B007NZS9SA,"[1.1794482, 0.4733188, 0.2671187, 1.8652414, 1..."
B00805IQO4,"[1.8137532, 0.012319085, 0.7137721, 3.0691266,..."
B008064WEQ,"[1.6741164, 0.82466173, 0.30143452, 1.160228, ..."
B009GEO1RY,"[0.97002316, 0.95374113, 0.032715112, 1.867379..."
B00A60XKZ6,"[2.019946, 0.1974048, 1.6857275, 1.2666014, 0...."
B00E9ECXGC,"[0.39486468, 0.37945437, 0.4299356, 1.698851, ..."


In [79]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartModel.from_pretrained('facebook/bart-base')

model.to(device)

c:\Users\Iga Miller\OneDrive\Pulpit\Studia_mgr_SI\Semestr2\Sys_rek\l3n-wstep-do-projektu-team1\.venv\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Iga Miller\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


BartModel(
  (shared): Embedding(50265, 768, padding_idx=1)
  (encoder): BartEncoder(
    (embed_tokens): Embedding(50265, 768, padding_idx=1)
    (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
    (layers): ModuleList(
      (0-5): 6 x BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layernorm_embedding): La

In [80]:
cardigan_items = cardigan_items[pd.notna(cardigan_items['description'])]

In [3]:
def text_embedding(text, max_token_size=1024):

    inputs = tokenizer(text[0], return_tensors="pt").to(device)

    inputs['input_ids'] = inputs['input_ids'][:, :max_token_size]
    inputs['attention_mask'] = inputs['attention_mask'][:, :max_token_size]

    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state

    return last_hidden_states.detach().cpu().numpy()[0][0]

# cardigan_items["Embedding"] = cardigan_items["description"].progress_apply(text_embedding)

In [82]:
cardigan_items

,category,description,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,price,also_buy,also_view,fit,details,main_cat,Embedding
0,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...",[Long sleeve handkerchief cardigan with pleath...,Derek Heart Girls' L/SLV Handkerchief Cardigan...,Derek Heart,"[95% Polyester, 5% Spandex, Imported, Machine ...","3,405,822inClothing,ShoesJewelry(",5 star,B00079R2MS,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,"[2.7669456, 2.8255193, 0.91699034, 7.971739, 0..."
1,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...",[Big girl cotton sweater cardigan with embroid...,Hartstrings Big Girls' Big Girl Sweater Cardig...,NaN,"[100% Cotton, Imported, Machine Wash, Front fl...","10,031,302inClothing,ShoesJewelry(",5 star,B000FIJ4TA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2.0541883, 2.8982399, 1.1833879, 7.463404, 1...."
2,"[Clothing, Shoes & Jewelry, Women, Clothing, C...",[This cardigan style sweater has a tie front c...,Lilo Maternity Tie Front Sweater (Pink),Lilo Maternity,"[Cotton, long sleeves, pointelle design, cardi...","16,723,371inClothing,ShoesJewelry(",5 star5 star (0%),B0016OSWK4,NaN,NaN,$40.14 - $61.00,NaN,NaN,NaN,NaN,NaN,"[2.8244548, 2.522948, 1.0347207, 7.398084, 0.7..."
3,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...","[Big girl cotton sweater cardigan, button down...",Hartstrings Big Girls' Big Sweater Cardigan,NaN,"[100% Cotton, Imported, Machine Wash, 100-perc...","9,199,644inClothing,ShoesJewelry(",5 star,B001CJQQSS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2.3283386, 2.8466587, 1.2076204, 7.1895013, 0..."
5,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",[Solid long sleeve hooded point Elle open fron...,OneWorld Women's Long Sleeve Pointelle Duster ...,OneWorld,"[100% Acrylic, Imported, Hand Wash, Hooded, Op...","3,531,727inClothing,ShoesJewelry(",5 star5 star (0%),B002EI5I9O,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,[B01E0FPAAK],NaN,NaN,NaN,NaN,"[2.4088006, 2.699317, 0.8265918, 7.563782, 1.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...",[Open front long sleeve shawl collar cozy with...,Derek Heart Big Girls' Long Sleeve Shawl Colla...,Derek Heart,"[100% Acrylic, Imported, Hand Wash, Fringe tri...","1,391,711inClothing,ShoesJewelry(",5 star,B01H7S23T8,NaN,NaN,$25.40,NaN,"[B017UVWLWQ, B01GQET93W, B07G2YZBWZ, B07CHD5HK...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[2.6011631, 2.7155693, 1.5041643, 7.890296, 0...."
1036,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...","[Long sleeve open front marled sweater cozy , ...",Derek Heart Big Girls' Long Sleeve Open Front ...,Derek Heart,"[100% Acrylic, Imported, Machine Wash, Dress, ...","1,258,383inClothing,ShoesJewelry(",5 star,B01H7S1WWW,NaN,NaN,NaN,"[B0134AB8XA, B016KX88S6, B074GZJZJJ, B01529O7I...",NaN,"class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[2.5779252, 2.9707854, 1.1147796, 7.6659045, 1..."
1037,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",[This cardigan by Denim & Supply features wool...,Denim & Supply Ralph Lauren Womens Boyfriend W...,Polo Ralph Lauren,[23%Acrylic/22%Nylon/20%Wool/20%Mohair/15%Visc...,"11,453,690inClothing,ShoesJewelry(",5 star5 star (0%),B01H7V69UO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,"[2.6039479, 2.3991945, 0.908189, 7.3224545, 0...."
1039,"[Clothing, Shoes & Jewelry, Girls, Clothing, S...",[A pretty knit design and trendy fringe trim o...,Epic Threads Girls' Knit Fringe Sweater,Epic Threads,[Shipping Information:\n \n...,"4,498,462inClothing,ShoesJewelry(",5 star5 star (0%),B01HEAYIA6,NaN,NaN,$16.30 - $20.31,NaN,"[B07CHD5HKV, B01LW7T4FP, B0157QWYZU, B07G2YZBW...",NaN,NaN,NaN,"[3.1346216, 2.142068, 1.701806, 7.6398816, 1.2..."


In [83]:
print(len(cardigan_items[pd.notna(cardigan_items['imageURL'])]))

252


In [84]:
merged_df = pd.merge(reviews, cardigan_items, on='asin')

In [85]:
merged_df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,...,date,imageURL,imageURLHighRes,price,also_buy,also_view,fit,details,main_cat,Embedding
0,5.0,True,"01 5, 2015",A2150DC5GKC9FL,B003NGLL0G,"{'Size:': ' Big Girls', 'Color:': ' Navy-ldda'}",Susan Alexander,My daughter goes to a charter school and they ...,Great Quality!,1420416000,...,5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$11.52 - $17.99,"[B07CMT5M7N, B007WZTHQE, B00XBBAH2G, B00CLBH3U...","[B00XBBAH2G, B007WZTHQE, B07CMT5M7N, B00CLBH3U...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[3.2996867, 2.5486076, 1.3442816, 8.33924, 1.1..."
1,5.0,True,"11 13, 2014",A1K1QAK7BBI4RS,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-he...",Tammy Smith,fits perfect!,Five Stars,1415836800,...,5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$11.52 - $17.99,"[B07CMT5M7N, B007WZTHQE, B00XBBAH2G, B00CLBH3U...","[B00XBBAH2G, B007WZTHQE, B07CMT5M7N, B00CLBH3U...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[3.2996867, 2.5486076, 1.3442816, 8.33924, 1.1..."
2,5.0,True,"10 13, 2014",A2GG708F913A7Z,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ld...",Theresa,"Recommend this seller,Has good clothes.","Very nice Sweater,",1413158400,...,5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$11.52 - $17.99,"[B07CMT5M7N, B007WZTHQE, B00XBBAH2G, B00CLBH3U...","[B00XBBAH2G, B007WZTHQE, B07CMT5M7N, B00CLBH3U...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[3.2996867, 2.5486076, 1.3442816, 8.33924, 1.1..."
3,4.0,True,"08 27, 2014",A1C4XG5FL8D1UC,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ld...",Rosanna,Very satisfied with the quality.,Four Stars,1409097600,...,5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$11.52 - $17.99,"[B07CMT5M7N, B007WZTHQE, B00XBBAH2G, B00CLBH3U...","[B00XBBAH2G, B007WZTHQE, B07CMT5M7N, B00CLBH3U...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[3.2996867, 2.5486076, 1.3442816, 8.33924, 1.1..."
4,5.0,True,"12 8, 2017",AJHPW0WZNVQQ0,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ah...",Erin Snyder,Cute little sweater. My daughter wears one nea...,Quality product,1512691200,...,5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$11.52 - $17.99,"[B07CMT5M7N, B007WZTHQE, B00XBBAH2G, B00CLBH3U...","[B00XBBAH2G, B007WZTHQE, B07CMT5M7N, B00CLBH3U...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[3.2996867, 2.5486076, 1.3442816, 8.33924, 1.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,5.0,True,"09 29, 2017",A2U9SM8D8PQS4P,B01H7S1WWW,"{'Size:': ' Big Girls', 'Color:': ' Pink Potion'}",Mrs.mommyAmazon,Its acrylic but its soft enough that my daught...,Just right,1506643200,...,5 star,NaN,NaN,NaN,"[B0134AB8XA, B016KX88S6, B074GZJZJJ, B01529O7I...",NaN,"class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[2.5779252, 2.9707854, 1.1147796, 7.6659045, 1..."
1239,5.0,False,"08 19, 2017",A1XNRR2JF0ZMZJ,B01H7S1WWW,"{'Size:': ' Big Girls', 'Color:': ' Pink Potion'}",Adriana M.,Very nice and good quality,Five Stars,1503100800,...,5 star,NaN,NaN,NaN,"[B0134AB8XA, B016KX88S6, B074GZJZJJ, B01529O7I...",NaN,"class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[2.5779252, 2.9707854, 1.1147796, 7.6659045, 1..."
1240,5.0,True,"08 15, 2017",A3UCDL5VSXMG8,B01H7S1WWW,"{'Size:': ' Big Girls', 'Color:': ' Pink Potion'}",Pat,Cute,Five Stars,1502755200,...,5 star,NaN,NaN,NaN,"[B0134AB8XA, B016KX88S6, B074GZJZJJ, B01529O7I...",NaN,"class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[2.5779252, 2.9707854, 1.1147796, 7.6659045, 1..."
1241,5.0,True,"06 22, 2017",A1VTS9MQOL10GT,B01H7S1WWW,"

In [98]:
df = pd.merge(games_with_features,cardigan_items, on="asin")
df

,asin,image_features,category,description,title,brand,feature,rank,date,imageURL,imageURLHighRes,price,also_buy,also_view,fit,details,main_cat,Embedding
0,B003NGLL0G,"[1.264941, 1.1645646, 0.43767372, 1.2647002, 0...","[Clothing, Shoes & Jewelry, Girls, Clothing, S...","[Established in 1890, the U.S. Polo Assn. is t...",U.S. Polo Assn Little Girls' Big Button Hooded...,U.S. Polo Assn.,"[100% Acrylic, Imported, Machine Wash, Easy ca...","127,622inClothing,ShoesJewelry(",5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$11.52 - $17.99,"[B07CMT5M7N, B007WZTHQE, B00XBBAH2G, B00CLBH3U...","[B00XBBAH2G, B007WZTHQE, B07CMT5M7N, B00CLBH3U...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[3.2996867, 2.5486076, 1.3442816, 8.33924, 1.1..."
1,B005DSLTUE,"[2.276295, 0.8267818, 0.4272437, 1.5532677, 0....","[Clothing, Shoes & Jewelry, Boys, Clothing, Sw...","[A comfortable classic with a modern take, thi...",French Toast Boys' V-Neck Anti-Pill Cardigan,NaN,"[100% Acrylic, Imported, Machine Wash, 100% Ac...","36,824inClothing,ShoesJewelry(",5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$12.99 - $24.98,"[B00YBQLZR6, B00XBBAGQI, B01DDFQP98, B00XR6Z9R...","[B07BN3P6YC, B01K6R3QSE, B00KRJSBGO, B00J65R5Y...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[2.566035, 2.6181617, 0.96879077, 7.575275, 0...."
2,B005DSLU9Y,"[1.8207275, 0.6596862, 0.440968, 1.3359298, 0....","[Clothing, Shoes & Jewelry, Boys, Clothing, Sw...","[A comfortable classic with a modern take, thi...",French Toast Boys' V-Neck Anti-Pill Cardigan,NaN,"[100% Acrylic, Imported, Machine Wash, 100% Ac...","36,824inClothing,ShoesJewelry(",5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$12.99 - $24.98,"[B00YBQLZR6, B00XBBAGQI, B01DDFQP98, B00XR6Z9R...","[B07BN3P6YC, B01K6R3QSE, B00KRJSBGO, B00J65R5Y...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[2.566035, 2.6181617, 0.96879077, 7.575275, 0...."
3,B007NZS9SA,"[1.1794482, 0.4733188, 0.2671187, 1.8652414, 1...","[Clothing, Shoes & Jewelry, Boys, Clothing, Sw...","[Youth sized unisex cardigan sweater, 100-perc...",CLASSROOM Boys' Uniform Cardigan Sweater,Classroom Uniforms,"[100% Acrylic, Imported, Machine Wash, Button ...","312,553inClothing,ShoesJewelry(",5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$17.68 - $23.98,"[B00X9HI4AY, B007EHYM9W, B007EHYMEM, B01K6R3QS...","[B00X9HI4AY, B01K6R3QSE, B007EHYM9W, B07BN3P6Y...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[2.4860094, 2.4747205, 1.2398989, 7.80946, 1.0..."
4,B00805IQO4,"[1.8137532, 0.012319085, 0.7137721, 3.0691266,...","[Clothing, Shoes & Jewelry, Boys, Clothing, Sw...","[For over 50 years, has been a leader in provi...",Genuine Little Boys' Sweater (More Styles Avai...,Genuine,"[100% Acrylic, Imported, Machine Wash, Soft an...","126,454inClothing,ShoesJewelry(",5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$9.99 - $14.99,"[B007QQ2MSO, B00YBQLZR6, B00X9HI4AY, B07BN3P6Y...","[B00X9HI4AY, B07BN3P6YC, B00YBQLZR6, B007QQ2MS...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"[2.9252636, 2.729303, 1.3336306, 7.6647196, 1...."
5,B008064WEQ,"[1.6741164, 0.82466173, 0.30143452, 1.160228, ...","[Clothing, Shoes & Jewelry, Girls, Clothing, S...","[For over 50 years, has been a leader in provi...",Genuine Girls' Sweater (More Styles Available),Genuine,"[100% Acrylic, Imported, Machine Wash, Soft an...","812,724inClothing,ShoesJewelry(",5 star,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,$14.99 - $15.99,"[B00XBBAH2G, B007WZTHQE, B07D4GK21P, B00XBBAGQ...","[B01ENGAYG6, B00XBBAH2G, B00XBBAGQI, B073CG5FW...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,"

In [89]:
filtered_reviews

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,vote,Embedding
0,5.0,True,"01 5, 2015",A2150DC5GKC9FL,B003NGLL0G,"{'Size:': ' Big Girls', 'Color:': ' Navy-ldda'}",Susan Alexander,My daughter goes to a charter school and they ...,Great Quality!,1420416000,NaN,NaN,"[2.4930532, 2.8495164, 1.1018121, 9.236096, 1...."
1,5.0,True,"11 13, 2014",A1K1QAK7BBI4RS,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-he...",Tammy Smith,fits perfect!,Five Stars,1415836800,NaN,NaN,"[3.1477046, 2.5275111, 0.9611606, 7.6812153, 0..."
2,5.0,True,"10 13, 2014",A2GG708F913A7Z,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ld...",Theresa,"Recommend this seller,Has good clothes.","Very nice Sweater,",1413158400,NaN,NaN,"[2.2573414, 2.8311124, 1.0643506, 8.796387, 1...."
3,4.0,True,"08 27, 2014",A1C4XG5FL8D1UC,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ld...",Rosanna,Very satisfied with the quality.,Four Stars,1409097600,NaN,NaN,"[2.558468, 3.0108557, 0.92667335, 8.368293, 1...."
4,5.0,True,"12 8, 2017",AJHPW0WZNVQQ0,B003NGLL0G,"{'Size:': ' Little Girls', 'Color:': ' Navy-ah...",Erin Snyder,Cute little sweater. My daughter wears one nea...,Quality product,1512691200,NaN,NaN,"[2.510398, 2.6544993, 0.7903514, 9.203217, 1.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124,5.0,True,"12 25, 2016",A354E34YDN9B3H,B01H7S23T8,"{'Size:': ' Big Girls', 'Color:': ' Cream'}",Amazon Customer,Awesome but the sleeves are tight.,Five Stars,1482624000,NaN,NaN,"[2.398775, 3.0312915, 1.387836, 8.797474, 1.24..."
2125,3.0,True,"12 2, 2016",A2U9I0GKYC5ENM,B01H7S23T8,"{'Size:': ' Big Girls', 'Color:': ' Black Beau...",Ed Davin,Nice sweater. I bought if for a gift and I ho...,Nice sweater. I bought if for a gift and ...,1480636800,NaN,NaN,"[2.7374265, 2.874777, 1.26762, 9.159906, 1.275..."
2126,5.0,True,"10 30, 2017",ASTHAOG1RBSWI,B01HCWDN3Y,"{'Size:': ' 3-4 Years', 'Color:': ' Grey'}",JMS,Perfect weight to wear over a shirt,Five Stars,1509321600,NaN,NaN,"[2.7698202, 2.9785373, 1.2093284, 8.927042, 1...."
2127,5.0,True,"10 22, 2017",A6GOZT13KZ1FZ,B01HCWDN3Y,"{'Size:': ' 18-24 Months', 'Color:': ' Pink'}",Trina,Cute sweater. Perfect fit. Good quality for th...,Loveeee it,1508630400,NaN,NaN,"[2.510398, 2.6544993, 0.7903514, 9.203217, 1.3..."


In [100]:
filtered_reviews['Embedding_reviews'] = filtered_reviews['reviewText'].progress_apply(text_embedding)

  0%|          | 0/2125 [00:00<?, ?it/s]

C:\Users\Iga Miller\AppData\Local\Temp\ipykernel_36872\1736675114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reviews['Embedding_reviews'] = filtered_reviews['reviewText'].progress_apply(text_embedding)


In [103]:
merged_df = df.merge(filtered_reviews[['asin', 'Embedding_reviews']], on='asin', how='inner')

merged_df.to_pickle('Merged_Metadata.pkl')

In [1]:
import pyarrow.feather as feather

df = feather.read_feather('data_train.feather')

In [4]:
df

,overall,reviewerID,asin
0,3.0,AZ1119BK1WNOP,B00007FFL9
1,5.0,A1VQL3LLOS1ZVX,B00007FFL9
2,5.0,A30WNM7RK2Z2VG,B00007FPNN
3,5.0,A30WOXPI820KFJ,B0000B35DA
4,5.0,A1U8VI6I2MFEU8,B0000B35DA
...,...,...,...
141008,4.0,A1MUFKXECGHPNR,B01HII35LC
141009,5.0,AVPZO0KSI5NBS,B01HJ79A4S
141010,5.0,A3SV0PTMJ54B0Z,B01HJC17Y4
141011,5.0,APDCPF4R6188U,B01HJC17Y4
